In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io
mat1 = scipy.io.loadmat('data/d0417-1.mat')
mat2 = scipy.io.loadmat('data/d0417-2.mat')
mat3 = scipy.io.loadmat('data/d0417-3.mat')
mat4 = scipy.io.loadmat('data/d0417-4.mat')
mat5 = scipy.io.loadmat('data/d0417-5.mat')
mat6 = scipy.io.loadmat('data/d0417-6.mat')
mat7 = scipy.io.loadmat('data/d0417-7.mat')
mat8 = scipy.io.loadmat('data/d0417n.mat')

TR=np.concatenate([mat8['d1'], mat8['d2'], mat8['d3'], mat8['d7'], mat8['d8']])
TS=np.concatenate([mat8['d4'], mat8['d5']])

TS6 = mat8['d6']
TS7 = mat8['d7']
TS8 = mat8['d8']
TS9 = mat8['d9']
print("Training set: ", TR.shape, "Test set: ", TS.shape)

('Training set: ', (300, 8204), 'Test set: ', (120, 8204))


In [2]:
sam_idx = np.linspace(0,8191, 2048, dtype='int8')
TR_sam=TR[:,sam_idx]
TS_sam=TS[:,sam_idx]

#### DEFINE NETWROK

In [3]:
device_type = "/cpu:0"
with tf.device(device_type):
    n_input=8192
    n_output = 12


    weights = {
        'wc1' : tf.get_variable("wc_1", shape=[2, 7, 1, 1], initializer=tf.contrib.layers.xavier_initializer()),
        'wc2' : tf.get_variable("wc_2", shape=[2, 7, 1, 3], initializer=tf.contrib.layers.xavier_initializer()),
        'wc3' : tf.get_variable("wc_3", shape=[2, 7, 3, 3], initializer=tf.contrib.layers.xavier_initializer()),
        'wd1' : tf.get_variable("wc_4", shape=[(int)(4096/16)*1, 32], initializer=tf.contrib.layers.xavier_initializer()),
        'wd2' : tf.get_variable("wd_2", shape=[32, n_output], initializer=tf.contrib.layers.xavier_initializer())
    }

    biases = {
        'bc1' : tf.Variable(tf.random_normal([1], stddev=0.1)),
        'bc2' : tf.Variable(tf.random_normal([14], stddev=0.1)),
        'bc3' : tf.Variable(tf.random_normal([21], stddev=0.1)),
        'bd1' : tf.Variable(tf.random_normal([32], stddev=0.1)),
        'bd2' : tf.Variable(tf.random_normal([n_output], stddev=0.1))
    }

In [4]:
device_type = "/cpu:0"
with tf.device(device_type):
    def conv_stap(_input, _w, _b, _keepratio):
        _input_r = tf.reshape(_input, shape=[-1, 2, 4096, 1])
        _pool0 = tf.nn.avg_pool(_input_r, ksize=[1,1,4,1], strides=[1,1,4,1], padding='SAME')
        _padded = tf.pad(_pool0, [[0,0],[0,0],[3,3],[0,0]], "CONSTANT")
        _conv1 = tf.nn.conv2d(_padded, _w['wc1'],strides=[1,1,1,1], padding='VALID')
        
        _mean, _var = tf.nn.moments(_conv1, [0,1,2])
        _conv1_n = tf.nn.batch_normalization(_conv1, _mean, _var, 0, 1, 0.0001)
        
        _conv1_r = tf.nn.relu(_conv1)
        _pool1 = tf.nn.max_pool(_conv1_r, ksize=[1,1,4,1], strides=[1,1,4,1], padding='SAME')
        _dr1   = tf.nn.dropout(_pool1, _keepratio)

        _dense = tf.reshape(_dr1, [-1, _w['wd1'].get_shape().as_list()[0]])

        _fc1 = tf.add(tf.matmul(_dense, _w['wd1']), _b['bd1'])
        _fc1_relu = tf.nn.relu(_fc1)
        _fc_dr1 = tf.nn.dropout(_fc1_relu, _keepratio)

        _out = tf.nn.softmax(tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2']))

        out = {
            'input_r': _input_r, 'conv1': _conv1, 'pool0': _pool0, 'pool1': _pool1
            ,'conv1_n':_conv1_n, 'conv1_r':_conv1_r, 'dense': _dense
            , 'dr1': _dr1, 'fc1': _fc1, 'fc1_relu':_fc1_relu, 'fc_dr1': _fc_dr1, 'out': _out
            , 'mean': _mean, 'var': _var
        }
        return out
    
def lrelu(x, alpha=0.1, max_value=None):
    negative_part=tf.nn.relu(-x)
    x=tf.nn.relu(x)
    
    if max_value is not None:
        x=tf.clip_by_value(x, tf.cast(0., dtype='float'), tf.cast(max_value, dtype='float'))
    
    x -= tf.constant(alpha, dtype='float') * negative_part
    return x



# DEFINE FUNCTIONS

In [5]:

with tf.device(device_type):
    x = tf.placeholder(tf.float32, [None, n_input])
    y = tf.placeholder(tf.float32, [None, n_output])
    keepratio = tf.placeholder(tf.float32)

    pred = conv_stap(x, weights, biases, keepratio)['out']
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    WEIGHT_DECAY_FACTOR = 0.0001
    l2_loss = tf.add_n([tf.nn.l2_loss(v)
                       for v in tf.trainable_variables()])
    cost = cost + WEIGHT_DECAY_FACTOR*l2_loss
    optm = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)
    corr = tf.equal(tf.argmax(pred,1), tf.argmax(y,1)) # count corrects
    accr = tf.reduce_mean(tf.cast(corr, tf.float32))
    init = tf.global_variables_initializer()


In [6]:
with tf.device(device_type):
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    sess.run(init)
    saver = tf.train.Saver()

# LEARNING

In [7]:
step = 105
idx1 = np.random.choice(105, size=20, replace=False)
idx2 = idx1+step
idx3 = idx1+step*2
idx4 = idx1+step*3
idx5 = idx1+step*4
idx6 = idx1+step*5
idx7 = idx1+step*6
idx8 = idx1+step*7
idx9 = idx1+step*8
idx10 = idx1+step*9
idx11 = idx1+step*10
idx12 = idx1+step*11
idx = np.concatenate([idx1,idx2,idx3,idx4,idx5,idx6,idx7,idx8,idx9,idx10,idx11,idx12])
idx


array([   8,  104,    3,   68,    1,  103,   72,   46,   51,   31,   73,
         28,   48,   34,   90,   58,   16,   20,   87,   38,  113,  209,
        108,  173,  106,  208,  177,  151,  156,  136,  178,  133,  153,
        139,  195,  163,  121,  125,  192,  143,  218,  314,  213,  278,
        211,  313,  282,  256,  261,  241,  283,  238,  258,  244,  300,
        268,  226,  230,  297,  248,  323,  419,  318,  383,  316,  418,
        387,  361,  366,  346,  388,  343,  363,  349,  405,  373,  331,
        335,  402,  353,  428,  524,  423,  488,  421,  523,  492,  466,
        471,  451,  493,  448,  468,  454,  510,  478,  436,  440,  507,
        458,  533,  629,  528,  593,  526,  628,  597,  571,  576,  556,
        598,  553,  573,  559,  615,  583,  541,  545,  612,  563,  638,
        734,  633,  698,  631,  733,  702,  676,  681,  661,  703,  658,
        678,  664,  720,  688,  646,  650,  717,  668,  743,  839,  738,
        803,  736,  838,  807,  781,  786,  766,  8

In [9]:
with tf.device(device_type):
    training_epoch = 1000
    display_step = 100
    batch_size = 180
       
    for epoch in range(training_epoch):
        avg_cost = 0.
        num_batch = int(180/batch_size)
        for i in range(num_batch):
            randidx = np.random.randint(180, size=batch_size)
            
            batch_xs = TR[:,0:8192]
            batch_ys = TR[:,8192:8204]

            #training
            sess.run(optm, feed_dict={x: batch_xs, y: batch_ys, keepratio: 0.5})
            #Compute average loss
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keepratio:1.})

        if epoch % display_step == 0 or epoch == training_epoch-1:

            print("Epoch: %03d/%03d cost: %.4f" % (epoch, training_epoch, avg_cost))
            training_acc = sess.run(accr, feed_dict={x: TR[:,0:8192], y: TR[:,8192:8204], keepratio:1.})
            test_acc     = sess.run(accr, feed_dict={x: TS[:,0:8192], y: TS[:,8192:8204], keepratio:1.})
            #acc = (training_acc*50 + test_acc*20 )/70
            print("TR acc : %.3f, TS acc : %.3f" % (training_acc, test_acc))
            #print("Test acc : %.3f" %(acc))


Epoch: 000/1000 cost: 1.7122
TR acc : 0.993, TS acc : 0.825
Epoch: 100/1000 cost: 1.7053
TR acc : 0.993, TS acc : 0.833
Epoch: 200/1000 cost: 1.7019
TR acc : 0.993, TS acc : 0.825
Epoch: 300/1000 cost: 1.6971
TR acc : 0.993, TS acc : 0.842
Epoch: 400/1000 cost: 1.6938
TR acc : 0.993, TS acc : 0.833
Epoch: 500/1000 cost: 1.6922
TR acc : 0.993, TS acc : 0.850
Epoch: 600/1000 cost: 1.6924
TR acc : 0.993, TS acc : 0.842
Epoch: 700/1000 cost: 1.6905
TR acc : 0.993, TS acc : 0.850
Epoch: 800/1000 cost: 1.6896
TR acc : 0.993, TS acc : 0.833
Epoch: 900/1000 cost: 1.6883
TR acc : 0.993, TS acc : 0.850
Epoch: 999/1000 cost: 1.6882
TR acc : 0.993, TS acc : 0.850


# STORE PARAMETERS

In [16]:
save_path = saver.save(sess, "parameters/model.ckpt")

# RESTORE PARAMETERS

In [36]:
saver.restore(sess, "parameters/model.ckpt")

# ACC CLASS BY CLASS

In [58]:
DS = np.concatenate([TS])
max_index = np.ndarray.argmax(DS[:,8192:],axis=1)
c01_index = np.where(max_index==0)
c02_index = np.where(max_index==1)
c03_index = np.where(max_index==2)
c04_index = np.where(max_index==3)
c05_index = np.where(max_index==4)
c06_index = np.where(max_index==5)
c07_index = np.where(max_index==6)
c08_index = np.where(max_index==7)
c09_index = np.where(max_index==8)
c10_index = np.where(max_index==9)
c11_index = np.where(max_index==10)
c12_index = np.where(max_index==11)
acc01 = sess.run(accr, feed_dict={x:DS[c01_index,0:8192][0,:,:], y:DS[c01_index,8192:8204][0,:,:], keepratio:1.})
acc02 = sess.run(accr, feed_dict={x:DS[c02_index,0:8192][0,:,:], y:DS[c02_index,8192:8204][0,:,:], keepratio:1.})
acc03 = sess.run(accr, feed_dict={x:DS[c03_index,0:8192][0,:,:], y:DS[c03_index,8192:8204][0,:,:], keepratio:1.})
acc04 = sess.run(accr, feed_dict={x:DS[c04_index,0:8192][0,:,:], y:DS[c04_index,8192:8204][0,:,:], keepratio:1.})
acc05 = sess.run(accr, feed_dict={x:DS[c05_index,0:8192][0,:,:], y:DS[c05_index,8192:8204][0,:,:], keepratio:1.})
acc06 = sess.run(accr, feed_dict={x:DS[c06_index,0:8192][0,:,:], y:DS[c06_index,8192:8204][0,:,:], keepratio:1.})
acc07 = sess.run(accr, feed_dict={x:DS[c07_index,0:8192][0,:,:], y:DS[c07_index,8192:8204][0,:,:], keepratio:1.})
acc08 = sess.run(accr, feed_dict={x:DS[c08_index,0:8192][0,:,:], y:DS[c08_index,8192:8204][0,:,:], keepratio:1.})
acc09 = sess.run(accr, feed_dict={x:DS[c09_index,0:8192][0,:,:], y:DS[c09_index,8192:8204][0,:,:], keepratio:1.})
acc10 = sess.run(accr, feed_dict={x:DS[c10_index,0:8192][0,:,:], y:DS[c10_index,8192:8204][0,:,:], keepratio:1.})
acc11 = sess.run(accr, feed_dict={x:DS[c11_index,0:8192][0,:,:], y:DS[c11_index,8192:8204][0,:,:], keepratio:1.})
acc12 = sess.run(accr, feed_dict={x:DS[c12_index,0:8192][0,:,:], y:DS[c12_index,8192:8204][0,:,:], keepratio:1.})
print("acc 01 : %.3f \nacc 02 : %.3f \nacc 03 : %.3f \nacc 04 : %.3f \nacc 05 : %.3f \nacc 06 : %.3f \nacc 07 : %.3f \nacc 08 : %.3f \nacc 09 : %.3f \nacc 10 : %.3f \nacc 11 : %.3f \nacc 12 : %.3f " 
      %(acc01, acc02, acc03, acc04, acc05, acc06, acc07, acc08, acc09, acc10, acc11, acc12))


acc 01 : 1.000 
acc 02 : 1.000 
acc 03 : 0.700 
acc 04 : 1.000 
acc 05 : 0.900 
acc 06 : 0.900 
acc 07 : 1.000 
acc 08 : 0.500 
acc 09 : 0.700 
acc 10 : 1.000 
acc 11 : 1.000 
acc 12 : 1.000 


# SAVE PARAMETERS FOR ANDROID

In [64]:
wc = sess.run(weights['wc1'])
wf1 = sess.run(weights['wd1'])
wf2 = sess.run(weights['wd2'])
bf1 = sess.run(biases['bd1'])
bf2 = sess.run(biases['bd2'])
Weights = {
    'wc':wc,
    'wf1':wf1,
    'wf2':wf2,
    'bf1':bf1,
    'bf2':bf2,
}
smat2 = scipy.io.savemat('data/savemat1.mat', Weights)

print(wc.shape)

(2, 7, 1, 1)


f = open("data/save1.txt", 'weights')
f.close()

# TEST

In [10]:
TSset = TS6
t1 = sess.run(pred, feed_dict={x:TSset[:,:8192], keepratio:1.} )
max_index = np.ndarray.argmax(t1,axis=1)

ntest_acc=sess.run(accr, feed_dict={x:TSset[:,0:8192], y:TSset[:,8192:8204], keepratio:1.})
print(ntest_acc)

print("1: ", max_index[0:5] +1)
print("2: ", max_index[5:10] +1)
print("3: ", max_index[10:15] +1)
print("4: ", max_index[15:20] +1)
print("5: ", max_index[20:25] +1)
print("6: ", max_index[25:30] +1)
print("7: ", max_index[30:35] +1)
print("8: ", max_index[35:40] +1)
print("9: ", max_index[40:45] +1)
print("10: ", max_index[45:50] +1)
print("11: ", max_index[50:55] +1)
print("12: ", max_index[55:60] +1)


0.983333
('1: ', array([1, 6, 1, 1, 1]))
('2: ', array([2, 2, 2, 2, 2]))
('3: ', array([3, 3, 3, 3, 3]))
('4: ', array([4, 4, 4, 4, 4]))
('5: ', array([5, 5, 5, 5, 5]))
('6: ', array([6, 6, 6, 6, 6]))
('7: ', array([7, 7, 7, 7, 7]))
('8: ', array([8, 8, 8, 8, 8]))
('9: ', array([9, 9, 9, 9, 9]))
('10: ', array([10, 10, 10, 10, 10]))
('11: ', array([11, 11, 11, 11, 11]))
('12: ', array([12, 12, 12, 12, 12]))
